# 04. 텍스트 분류 실습 - 20 뉴스그룹 분류

p481(500) ~

<br>

사이킷런이 내부에 가지고 있는 예제 데이터인 "20 뉴스그룹 데이터 세트"를 이용 텍스트 분류 적용.

텍스트 분류

- 특정 문서의 분류를 학습 데이터를 통해 학습해 모델 생성
- 이 학습 모델을 이용해 다른 문서의 분류를 예측

사이킷런은 `fetch_20newsgroups()` API를 이용해 뉴스그룹의 분류를 수행해 볼 수 있는 예제 데이터를 제공.

텍스트를 피처 벡터화로 변환하면 일반적으로 희소 행렬 형태가 된다.

이러한 희소 행렬에 분류를 효과적으로 잘 처리할 수 있는 알고리즘

- 로지스틱 회귀
- 선형 서포트 벡터 머신
- 나이즈 베이즈

텍스트를 기반으로 분류를 수행할 때는 먼저 텍스트를 정규화한 뒤 피처 벡터화를 적용.

그리고 그 이후에 적합한 머신러닝 알고리즘을 적용해 분류를 학습/예측/평가.

이번 절에서는 카운트 기반과 TF-IDF 기반의 벡터화를 차례로 적용해 예측 성능을 비교.

피처 벡터화를 위한 파라미터와 `GridSearchCV` 기반의 하이퍼 파라미터 튜닝, 사이킷런의 `Pipeline` 객체를 통해 피처 벡터화 파라미터와 `GridSearchCV` 기반의 하이퍼 파라미터 튜닝을 한꺼번에 하는 방법 소개

<br>

## 4.1 텍스트 정규화

In [1]:
from sklearn.datasets import fetch_20newsgroups

news_data = fetch_20newsgroups(subset='all', random_state=156)

<br>

`fetch_20newsgroups()`는 파이썬 딕셔너리와 유사한 `Bunch` 객체를 반환.

In [2]:
print(news_data.keys())

dict_keys(['data', 'filenames', 'target_names', 'target', 'DESCR'])


`filenames` : `fetch_20newsgroups()` API가 인터넷에서 내려받아 로컬 컴퓨터에 저장하는 디렉터리와 파일명을 지칭

<br>

`Target` 클래서 구성 확인

In [3]:
import pandas as pd

print('target 클래스의 값과 분포도 \n', pd.Series(news_data.target).value_counts().sort_index())
print('target 클래스의 이름들 \n', news_data.target_names)

target 클래스의 값과 분포도 
 0     799
1     973
2     985
3     982
4     963
5     988
6     975
7     990
8     996
9     994
10    999
11    991
12    984
13    990
14    987
15    997
16    910
17    940
18    775
19    628
dtype: int64
target 클래스의 이름들 
 ['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']


<br>

`Target` 클래스의 값은 0부터 19까지 20개로 구성.

개별 데이터가 텍스트로 어떻게 구성돼 있는 지 데이터 한 개만 추출하여 확인

In [4]:
print(news_data.data[0])

From: egreen@east.sun.com (Ed Green - Pixel Cruncher)
Subject: Re: Observation re: helmets
Organization: Sun Microsystems, RTP, NC
Lines: 21
Distribution: world
Reply-To: egreen@east.sun.com
NNTP-Posting-Host: laser.east.sun.com

In article 211353@mavenry.altcit.eskimo.com, maven@mavenry.altcit.eskimo.com (Norman Hamer) writes:
> 
> The question for the day is re: passenger helmets, if you don't know for 
>certain who's gonna ride with you (like say you meet them at a .... church 
>meeting, yeah, that's the ticket)... What are some guidelines? Should I just 
>pick up another shoei in my size to have a backup helmet (XL), or should I 
>maybe get an inexpensive one of a smaller size to accomodate my likely 
>passenger? 

If your primary concern is protecting the passenger in the event of a
crash, have him or her fitted for a helmet that is their size.  If your
primary concern is complying with stupid helmet laws, carry a real big
spare (you can put a big or small head in a big helmet, bu

뉴스그룹 기사의 내용뿐만 아니라 뉴스그룹 제목, 작성자, 소속, 이메일 등의 다양한 정보를 가지고 있음.

이 중에서 내용을 제외하고 제목 등의 다른 정보는 제거.  
(제목과 소속, 이메일 주소 등의 헤더와 푸터 정보들은 뉴스그룹 분류의 `Target` 클래스 값과 유사한 데이터를 가지고 있는 경우가 많기 때문)

`remove` 파라미터를 이용하면 뉴스그룹 기사의 헤더(header), 푸터(footer) 등을 제거할 수 있다.

또한 `fetch_20newsgroups()`는 `subset` 파라미터를 이용해 학습 데이터 세트와 테스트 데이터 세트를 분리해 내려받을 수 있다.

In [5]:
from sklearn.datasets import fetch_20newsgroups

# subset='train' 으로 학습용 데이터만 추출
# remove=('headers', 'footers', 'quotes')로 내용만 추출
train_news = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'), random_state=156)

X_train = train_news.data
y_train = train_news.target

# subset='test' 으로 테스트 데이터만 추출
# remove=('headers', 'footers', 'quotes')로 내용만 추출
test_news = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'), random_state=156)

X_test = test_news.data
y_test = test_news.target

print('학습 데이터 크기 {0}, 테스트 데이터 크기 {1}'.format(len(train_news.data), len(test_news.data)))

학습 데이터 크기 11314, 테스트 데이터 크기 7532


<br>

## 4.2 피처 벡터화 변환과 머신러닝 모델 학습/예측/평가

학습 데이터 : 11,314개의 뉴스그룹 문서 (리스트 형태)  
테스트 데이터 : 7,532개의 뉴스그룹 문서 (리스트 형태)

<br>

### 4.2.1 피처 벡터화

`CountVectorizer`를 이용해 학습 데이터의 텍스트를 피처 벡터화 수행

테스트 데이터에 대해 피처 벡터화를 수행할 때 유의해야 할 점이 있음
  1. 테스트 데이터에서 `CountVectorizer`를 적용할 때는 반드시 **학습 데이터를 이용해 `fit()`이 수행된 `CountVectorizer` 객체를 이용**해 테스트 데이터를 변환(transform)해야 한다.
    - 그래야만 학습 시 설정된 `CountVectorizer`의 피처 개수와 테스트 데이터를 `CountVectorizer`로 변환할 피처 개수가 같아진다.
    - 테스트 데이터의 피처 벡터화는 학습 데이터에 사용된 `CountVectorizer` 객체 변수인 `cnt_vect.transform()`을 이용해 변환한다.
  2. 테스트 데이터 피처 벡터화 시 `fit_transform()`을 사용하면 안됨
    - `CountVectorizer.fit_transform()`을 수행하면 테스트 데이터 기반으로 `fit()`, `transform()` 되기 때문에 학습 시 사용된 피처 개수와 예측 시 사용할 피처 개수가 달라진다.

In [6]:
from sklearn.feature_extraction.text import CountVectorizer

# Count Vectorization으로 피처 벡터화 변환 수행
cnt_vect = CountVectorizer()
cnt_vect.fit(X_train, y_train)
X_train_cnt_vect = cnt_vect.transform(X_train)

# 학습 데이터로 fit()된 CountVecorizer를 이용해 테스트 데이터를 피처 벡터화 변환 수행
X_test_cnt_vect = cnt_vect.transform(X_test)

print("학습 데이터 텍스트의 CountVectorizer Shape: ", X_train_cnt_vect.shape)

학습 데이터 텍스트의 CountVectorizer Shape:  (11314, 101631)


학습 데이터를 `CountVectorizer`로 피처를 추출한 결과 11,314개의 문서에서 피처, 즉 단어가 101,631개로 만들어짐

<br>

### 4.2.2 뉴스 그룹 분류 예측

피처 벡터화된 데이터에 로지스틱 회귀를 적용해 뉴스그룹에 대한 분류를 예측

In [7]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# LogisticRegression을 이용해 학습/예측/평가 수행
lr_clf = LogisticRegression()
lr_clf.fit(X_train_cnt_vect, y_train)
pred = lr_clf.predict(X_test_cnt_vect)
print("CountVectorized Logistic Regression의 예측 정확도 : {0:.3f}".format(accuracy_score(y_test, pred)))

CountVectorized Logistic Regression의 예측 정확도 : 0.606


c:\users\mktinfo\.conda\envs\python36\lib\site-packages\sklearn\linear_model\_logistic.py:939: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html.
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Count 기반으로 피처 벡터화가 적용된 데이터 세트에 대한 로지스틱 회귀의 예측 정확도 : 0.606

<br>

### 4.2.3 TF-IDF 기반 벡터화 및 예측 모델 수행

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer

# TF-IDF 벡터화를 적용해 학습 데이터 세트와 테스트 데이터 세트 변환
tfidf_vect = TfidfVectorizer()
tfidf_vect.fit(X_train)
X_train_tfidf_vect = tfidf_vect.transform(X_train)
X_test_tfidf_vect = tfidf_vect.transform(X_test)

# LogisticRegression을 이용해 학습/예측/평가 수행
lr_clf = LogisticRegression()
lr_clf.fit(X_train_tfidf_vect, y_train)
pred = lr_clf.predict(X_test_tfidf_vect)
print("TF-IDF Logistic Regression의 예측 정확도 : {0:.3f}".format(accuracy_score(y_test, pred)))

TF-IDF Logistic Regression의 예측 정확도 : 0.674


TF-IDF가 단순 카운트 기반보다 훨씬 더 높은 예측 정확도(0.674 > 0.606)를 제공함.

일반적으로 문서 내에 텍스트가 많고 많은 문서를 가지는 텍스트 분석에는 카운트 벡터화보다는 TF-IDF 벡터화가 좋은 예측 결과를 도출한다.

<br>

### 4.2.4 텍스트 분석에서 머신러닝 모델 성능 향상 방법 (2가지)

1. 최적의 ML 알고리즘을 선택
2. 최상의 피처 전처리를 수행
  - 텍스트 정규화, Count/TF-IDF 기반 피처 벡터화를 어떻게 효과적으로 적용했는 지가 큰 영향을 미칠 수 있음

<br>

### 4.2.5 TF-IDF 벡터화 하이퍼 파라미터 조정

- 스톱워드 : `None` $\rightarrow$ `english`
- `ngram_range` : `(1,1)` $\rightarrow$ `(1,2)`
- `max_df = 300`

In [9]:
# stop words 필터링을 추가하고 ngram을 기본 (1,1)에서 (1,2)로 변경해 피처 벡터화 적용
tfidf_vect = TfidfVectorizer(stop_words='english',
                             ngram_range=(1,2),
                             max_df=300)

tfidf_vect.fit(X_train)
X_train_tfidf_vect = tfidf_vect.transform(X_train)
X_test_tfidf_vect = tfidf_vect.transform(X_test)

lr_clf = LogisticRegression()
lr_clf.fit(X_train_tfidf_vect, y_train)
pred = lr_clf.predict(X_test_tfidf_vect)
print("TF-IDF Vectorized Logistic Regression의 예측 정확도 : {0:.3f}".format(accuracy_score(y_test, pred)))

TF-IDF Vectorized Logistic Regression의 예측 정확도 : 0.692


예측 정확도 향상 (0.692 > 0.674)

<br>

### 4.2.6 `GridSearchCV` 이용 예측 모델 하이퍼 파라미터 최적화

로지스틱 회귀의 `C` 파라미터만 변경하면서 최적의 `C`값을 찾은 뒤 이 `C`값으로 학습된 모델에서 테스트 데이터로 예측해 성능을 평가

In [ ]:
from sklearn.model_selection import GridSearchCV

# 최적 C 값 도출 튜닝 수행. CV는 3 폴드 세트로 설정
params = {'C': [0.01, 0.1, 1, 5, 10]}
grid_cv_lr = GridSearchCV(lr_clf, param_grid=params, cv=3, scoring='accuracy', verbose=1)
grid_cv_lr.fit(X_train_tfidf_vect, y_train)
print("Logistic Regression best C parameter : ", grid_cv_lr.best_params_)

Fitting 3 folds for each of 5 candidates, totalling 15 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


로지스틱 회귀의 `C`가 10일 때 `GridSearchCV`의 교차 검증 테스트 세트에서 가장 좋은 예측 성능을 나타냄

In [ ]:
# 최적 C 값으로 학습된 grid_cv로 예측 및 정확도 평가
pred = grid_cv_lr.predict(X_test_tfidf_vect)
print("TF-IDF Vectorized Logistic Regression의 예측 정확도 : {0:.3f}".format(accuracy_score(y_test, pred)))

이를 테스트 데이터 세트에 적용해 약 0.888(> 0.692)으로 이전보다 약간 향상된 성능 수치가 됨

<br>

## 4.3 사이킷런 파이프라인(`Pipeline`) 사용 및 `GridSearchCV`와의 결합